In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import pivot_table
import json

In [ ]:
### convert data to regional view

In [138]:
pd.set_option('display.width', 1000)

In [139]:
horig = pd.read_csv("h_clean_2017.csv", header=0)

print(horig.shape)
horig.head(2)

(4550, 11)


,Unnamed: 0,ID,Geography,Year,Sex,Hours,WorkingPattern,Value,CV,lat,lon
0,60,E06000005,Darlington,2017,All,Basic,Full,40.0,3.8,54.535301,-1.56835
1,61,E06000005,Darlington,2017,All,Basic,Part,17.5,5.3,54.535301,-1.56835


In [140]:
print(f"Unique IDs: {len(horig['ID'].unique())}")

Unique IDs: 380


In [141]:
print(horig['Sex'].unique())
print(horig['Hours'].unique())
print(horig['WorkingPattern'].unique())

['All' 'Male' 'Female']
['Basic' 'Overtime']
['Full' 'Part']


In [142]:
horig = horig[horig.Sex == 'All']
horig = horig[horig.Hours == 'Basic']
horig = horig[horig.WorkingPattern == 'Full']

horig.shape

(380, 11)

## Append Region

In [143]:
region = pd.read_csv('Local_Authority_District_to_Region_December_2018_Lookup_in_England.csv')

print(region.shape)
region.head(2)

(326, 5)


,LAD18CD,LAD18NM,RGN18CD,RGN18NM,FID
0,E09000001,City of London,E12000007,London,1
1,E09000002,Barking and Dagenham,E12000007,London,2


In [144]:
region.RGN18NM.unique()

array(['London', 'North West', 'Yorkshire and The Humber', 'North East',
       'East of England', 'South East', 'East Midlands', 'South West',
       'West Midlands'], dtype=object)

In [145]:
region = region[['LAD18CD', 'RGN18NM']]
region.rename(columns={"LAD18CD":"ID", "RGN18NM":"Region"}, inplace=True)

region.head(2)

,ID,Region
0,E09000001,London
1,E09000002,London


In [146]:
hreg = horig.copy()
print(hreg.shape)

hreg = pd.merge(hreg, region, how='left', on=['ID'])
print(hreg.shape)

hreg.head(2)

(380, 11)
(380, 12)


,Unnamed: 0,ID,Geography,Year,Sex,Hours,WorkingPattern,Value,CV,lat,lon,Region
0,60,E06000005,Darlington,2017,All,Basic,Full,40.0,3.8,54.535301,-1.56835,North East
1,132,E06000001,Hartlepool,2017,All,Basic,Full,37.6,1.2,54.676201,-1.27023,North East


In [147]:
hreg = hreg[['ID', 'Value', 'Region']]

hreg.head(2)

,ID,Value,Region
0,E06000005,40.0,North East
1,E06000001,37.6,North East


In [148]:
for i in range(len(hreg)):
    if hreg.iloc[i, 0][0] == 'W':
        hreg.at[i, 'Region'] = 'Wales'
    elif hreg.iloc[i, 0][0] == 'S':
        hreg.at[i, 'Region'] = 'Scotland'

hreg.Region.value_counts()

South East                  67
East of England             47
East Midlands               40
North West                  39
South West                  37
London                      33
Scotland                    32
West Midlands               30
Wales                       22
Yorkshire and The Humber    21
North East                  12
Name: Region, dtype: int64

In [149]:
# national average == 38.3
all_pivot = pd.pivot_table(hreg, values='Value', index=['Region'], margins=True, aggfunc=np.mean)
all_pivot['Value'] = round(all_pivot['Value'], 1)

all_pivot.tail(1)

,Value
Region,
All,38.3


In [150]:
# confirm national average == 38.3
horig.Value.describe()

count    380.000000
mean      38.273684
std        0.700013
min       36.500000
25%       37.800000
50%       38.200000
75%       38.625000
max       41.600000
Name: Value, dtype: float64

In [160]:
h_pivot = pd.pivot_table(hreg, values='Value', index=['Region'], aggfunc=np.mean)
h_pivot['Value'] = round(h_pivot['Value'], 1)

h_pivot.head(2)

,Value
Region,
East Midlands,38.5
East of England,38.6


## Append Lat Lon

In [161]:
geo = pd.read_csv('NUTS_Level_1_January_2018_Ultra_Generalised_Clipped_Boundaries_in_the_United_Kingdom.csv')

print(geo.shape)
geo.head(2)

(12, 9)


,objectid,nuts118cd,nuts118nm,bng_e,bng_n,long,lat,st_areashape,st_lengthshape
0,1,UKC,North East (England),417313,600358,-1.72890,55.297031,8.609939e+09,6.575782e+05
1,2,UKD,North West (England),350015,506280,-2.77237,54.449451,1.418261e+10,1.063053e+06


In [162]:
def name_clean(x):
    if x[-10:] == ' (England)':
        return x[:-10]
    else:
        return x

In [163]:
geo.nuts118nm = geo.nuts118nm.apply(name_clean)
geo = geo[['nuts118cd', 'nuts118nm', 'lat', 'long']]
geo.rename(columns={"nuts118cd":"ID", "nuts118nm":"Region", "long":"lon"}, inplace=True)

geo.head()

,ID,Region,lat,lon
0,UKC,North East,55.297031,-1.72890
1,UKD,North West,54.449451,-2.77237
2,UKE,Yorkshire and The Humber,53.932640,-1.28712
3,UKF,East Midlands,52.795719,-0.84967
4,UKI,London,51.492271,-0.30864


In [164]:
print(h_pivot.shape)

h_pivot = pd.merge(h_pivot, geo, on=['Region'])
h_pivot.rename(columns={"Region":"Geography"}, inplace=True)
print(h_pivot.shape)

h_pivot.head(2)

(11, 1)
(11, 5)


,Geography,Value,ID,lat,lon
0,East Midlands,38.5,UKF,52.795719,-0.849670
1,East of England,38.6,UKH,52.240669,0.504146


In [166]:
# h_pivot.to_csv('h_region_2017.csv')